In [1]:
import os
import sys
import io
import openai
import anthropic
import gradio as gr
import subprocess

from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
print(f"openai api key is loaded: {openai_api_key[:8]}")
print(f"anthropic api key is loaded: {anthropic_api_key[:8]}")

GPT_MODEL = 'gpt-4o-mini'
CLAUDE_MODEL = 'claude-3-7-sonnet-latest'
print(f"using gpt model '{GPT_MODEL}'...")
print(f"using claude model '{CLAUDE_MODEL}...'")

SYS_PROMPT = """
You are coding translator. You get a piece of code in python and your task is to 
generate analogous code in C++. You respond just with a code ready to be executed.
No explanations required.
"""
print("setting system prompt...")

gpt = openai.OpenAI()
claude = anthropic.Anthropic()

py_code="""import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

openai api key is loaded: sk-proj-
anthropic api key is loaded: sk-ant-a
using gpt model 'gpt-4o-mini'...
using claude model 'claude-3-7-sonnet-latest...'
setting system prompt...


In [2]:
def stream_gpt(prompt):
    messages = [
        {'role': 'system', 'content': SYS_PROMPT},
        {'role': 'user', 'content': prompt}
    ]
    stream = gpt.chat.completions.create(
        model=GPT_MODEL,
        messages=messages,
        stream=True
    )

    for text in stream:
        chunk = text.choices[0].delta.content or ''
        yield chunk


def stream_claude(prompt):
    messages = [
        {'role': 'user', 'content': prompt}
    ]

    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=SYS_PROMPT,
        messages=messages
    )


    with result as stream:
        for text in stream.text_stream:
            yield text


In [3]:
def optimize(py_code, model):
    if model == 'gpt':
        stream = stream_gpt(py_code)
    elif model == 'Claude':
        stream = stream_claude(py_code)
    else:
        raise Exception(f'Model {model} is unknown')

    resp = ''
    for chunk in stream:
        resp += chunk
        
    yield resp.replace("```cpp\n", "").replace("```", "")

py_hard =  """# Be careful to support large number sizes
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

def execute_python(code):
    try:
        namespace = {}
        output = io.StringIO()
        sys.stdout = output
        exec(code, namespace)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()


def execute_cpp(code):
    with open("code.cpp", "w") as f:
        f.write(code)
    try:
        subprocess.run(["clang++", "-O3", "-std=c++17", "-o", "code", "code.cpp"], check=True)
        result = subprocess.run(["./code"], check=True, text=True, capture_output=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred: \n{e.stderr}"

css="""
.python {background-color: #FFFFCC;}
.cpp {background-color: #CCFFCC;}
"""


with gr.Blocks(css=css) as ui:
    with gr.Row():
        python_code = gr.Textbox(label='Python code', lines=10, value=py_hard)
        cpp_code = gr.Textbox(label='C++ code', lines=10)
    with gr.Row():
        model = gr.Dropdown(['Claude', 'gpt'])
        convert_btn = gr.Button('Convert')
    with gr.Row():
        python_run = gr.Button('Run Python')
        cpp_run = gr.Button('Run C++')
    with gr.Row():
        python_out = gr.TextArea(label='Python result', elem_classes="python")
        cpp_out = gr.TextArea(label='C++ result', elem_classes="cpp")

    convert_btn.click(optimize, inputs=[python_code, model], outputs=[cpp_code])
    python_run.click(execute_python, inputs=python_code, outputs=python_out)
    cpp_run.click(execute_cpp, inputs=cpp_code, outputs=cpp_out)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


code.cpp:7:102: warning: implicit conversion from 'unsigned long long' to 'uint32_t' (aka 'unsigned int') changes value from 4294967296 to 0 [-Wconstant-conversion]
uint32_t lcg_next(uint32_t& value, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = (1ULL << 32)) {
                                                                                             ~  ~~~~~^~~~~
1 warning generated.
code.cpp:7:105: warning: shift count >= width of type [-Wshift-count-overflow]
std::vector<uint32_t> lcg(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1U << 32) {
                                                                                                        ^  ~~
1 warning generated.
